<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, time, uuid, re, json, datetime
# 標準且清晰的匯入方式
import datetime as dt_module # 將整個 datetime 模組別名為 dt_module
from datetime import datetime as dt # 將 datetime 類別別名為 dt，用於呼叫 now()
from datetime import timedelta     # 將 timedelta 類別單獨匯入
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import pytz
from IPython.display import display, Markdown
from gspread_dataframe import set_with_dataframe, get_as_dataframe

import google.generativeai as genai

In [2]:
# Google Auth & Sheets
from google.colab import auth
import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
from google.colab import auth
auth.authenticate_user()

In [3]:
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
from google.colab import files
import os

In [4]:
print("請上傳您的 PDF 課表檔案...")
# 顯示上傳按鈕
uploaded = files.upload()

pdf_filename = None
if not uploaded:
  print("\n⚠️ 您沒有上傳任何檔案。")
else:
  # 取得上傳檔案的名稱
  pdf_filename = next(iter(uploaded))
  print(f"\n✅ 成功上傳檔案: '{pdf_filename}'")

  # 檢查是否為 PDF (簡易檢查)
  if not pdf_filename.lower().endswith('.pdf'):
    print(f"⚠️ 警告：您上傳的檔案 '{pdf_filename}' 不是 PDF 檔。")
%store pdf_filename

請上傳您的 PDF 課表檔案...


Saving export.pdf to export (2).pdf

✅ 成功上傳檔案: 'export (2).pdf'
Stored 'pdf_filename' (str)


In [5]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
# 警告：此處使用 'HW3' 作為 Secret 名稱，請確保您已設置
api_key = userdata.get('HW3')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

In [6]:
# 讀取上一個儲存格存的檔案名稱
try:
    %store -r pdf_filename

    if pdf_filename is None or not os.path.exists(pdf_filename):
        print(f"❌ 找不到檔案 '{pdf_filename}'，請重新執行上一個儲存格上傳。")
    else:
        print(f"準備處理檔案: '{pdf_filename}'")

        # 1. 上傳檔案到 Gemini File API
        print("正在上傳 PDF 至 Gemini File API...")
        gemini_file = genai.upload_file(path=pdf_filename,display_name="114-1 課程表 PDF")
        print(f"✅ 檔案上傳成功！(File URI: {gemini_file.uri})")

        # 2. 準備 Prompt
        prompt_text = f"""
        你是一位 helpful AI assistant。
        請分析我上傳的 PDF 課表檔案 (檔案名稱: {pdf_filename})。

        這份 PDF 課表是「國立臺灣師範大學114學年度第1學期課表」。
        授課教師是「蔡芸琤」。

        請幫我將這份課表，轉換成每周的紀錄。
        學期開學日是 2025/9/1，學期結束日是 2025/12/19。

        你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，
        不要有任何其他說明文字 (例如 "好的，這是..." 或 "```csv" 標記)。

        CSV 格式的欄位必須如下：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"

        - 「攜帶品」、「先讀章節」、「備註」這三欄因為 PDF 中沒有，請保持空白。
        - 「日期」範圍請涵蓋 2025/9/1 到 2025/12/19 之間所有有課的日期。
        - 請仔細解析 PDF 中的表格，找出所有課程、時間和地點。
        """

        # 3. 呼叫 Gemini 1.5 Pro 模型 (使用 Pro 處理 PDF 結構化效果更好)
        print("正在向 Gemini 2.5 Pro 提出請求 (這可能需要一點時間)...")
        model = genai.GenerativeModel('gemini-2.5-pro')

        # 發送包含「Prompt文字」和「上傳的檔案」的請求
        response = model.generate_content([prompt_text, gemini_file])

        # 4. 刪除在 Gemini 雲端上的檔案 (避免佔用空間)
        print("正在從 Gemini 雲端刪除暫存檔案...")
        genai.delete_file(gemini_file.name)
        print("✅ 暫存檔案已刪除。")

        # 5. 顯示結果並儲存
        csv_output = response.text.strip().replace("```csv\n", "").replace("```", "")
        print("\n--- 🤖 Gemini 產生的 CSV 內容 ---")
        print(csv_output)
        print("---------------------------------")

        # 6. 將 Gemini 產生的 CSV 內容存成一個檔案
        with open("gemini_schedule.csv", "w", encoding='utf-8') as f:
            f.write(csv_output)

        print("\n✅ 已將 Gemini 產生的結果儲存為 'gemini_schedule.csv'")

except NameError:
    print("❌ 找不到 'pdf_filename'。請先執行上一個儲存格來上傳檔案。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")
    # 如果發生錯誤，也嘗試刪除已上傳的檔案
    try:
        if 'gemini_file' in locals():
            genai.delete_file(gemini_file.name)
            print("已清理遠端暫存檔案。")
    except Exception as del_e:
        print(f"清理檔案時發生錯誤: {del_e}")

準備處理檔案: 'export (2).pdf'
正在上傳 PDF 至 Gemini File API...
✅ 檔案上傳成功！(File URI: https://generativelanguage.googleapis.com/v1beta/files/sz78akn5sko5)
正在向 Gemini 2.5 Pro 提出請求 (這可能需要一點時間)...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 6336.34ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 8671.30ms


正在從 Gemini 雲端刪除暫存檔案...
✅ 暫存檔案已刪除。

--- 🤖 Gemini 產生的 CSV 內容 ---
"日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
"2025/9/1","一","機械製造","08:10","12:10","科技系TC504教室","","",""
"2025/9/1","一","教育哲學(教)","13:20","15:10","教310","","",""
"2025/9/1","一","班級經營(教)","15:30","17:20","誠107","","",""
"2025/9/2","二","木工製造","08:10","12:10","科技系TC501教室","","",""
"2025/9/2","二","色彩學","13:20","16:20","科技系TB213教室","","",""
"2025/9/3","三","3D列印與創新應用","10:20","12:10","誠102","","",""
"2025/9/4","四","程式語言","09:10","12:10","科技系TB311教室","","",""
"2025/9/4","四","經典與情緒素養","13:20","15:10","綜210展覽廳","","",""
"2025/9/4","四","機構設計","17:30","20:25","科技系TC504教室","","",""
"2025/9/5","五","電腦輔助製圖","14:20","17:20","科技系TA509教室","","",""
"2025/9/8","一","機械製造","08:10","12:10","科技系TC504教室","","",""
"2025/9/8","一","教育哲學(教)","13:20","15:10","教310","","",""
"2025/9/8","一","班級經營(教)","15:30","17:20","誠107","","",""
"2025/9/9","二","木工製造","08:10","12:10","科技系TC501教室","","",""
"2025/9/9","二","色彩學","13:20","16:20","科技系TB213教室","",

In [7]:
# --- 1. 定義您的 Google Sheet 資訊 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing"
WORKSHEET_NAME = "HW6"
TIMEZONE = "Asia/Taipei"

In [8]:
# --- 2. 讀取 Gemini 產生的 CSV 檔案 ---
try:
    df_to_upload = pd.read_csv("gemini_schedule.csv")
    print(f"✅ 成功讀取 'gemini_schedule.csv'，共 {len(df_to_upload)} 筆資料。")

    # --- 3. 準備 gspread ---
    df_to_upload = df_to_upload.fillna('')

    # --- 4. 開啟 Google Sheet 和指定的分頁 ---
    print(f"正在開啟 Google Sheet (URL: {SHEET_URL})...")
    gsheets = gc.open_by_url(SHEET_URL)

    print(f"正在存取分頁: '{WORKSHEET_NAME}'...")
    try:
        # 嘗試取得分頁，如果不存在就建立一個新的
        worksheet = gsheets.worksheet(WORKSHEET_NAME)
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到分頁 '{WORKSHEET_NAME}'，將自動建立新分頁...")
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    # --- 5. 將 DataFrame 寫入工作表 ---
    print(f"正在將 {len(df_to_upload)} 筆資料寫入 '{WORKSHEET_NAME}' 分頁...")
    # set_with_dataframe 會自動處理清空與標頭寫入
    set_with_dataframe(worksheet, df_to_upload)

    print(f"\n✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '{WORKSHEET_NAME}' 分頁！")

except FileNotFoundError:
    print("❌ 錯誤：找不到 'gemini_schedule.csv' 檔案。")
    print("請確認您執行 Gemini API 的儲存格已成功產生這個檔案。")
except pd.errors.EmptyDataError:
    print("❌ 錯誤：'gemini_schedule.csv' 是空的，沒有資料可以上傳。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")

✅ 成功讀取 'gemini_schedule.csv'，共 160 筆資料。
正在開啟 Google Sheet (URL: https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing)...
正在存取分頁: 'HW6'...
正在將 160 筆資料寫入 'HW6' 分頁...

✅ 成功將 'gemini_schedule.csv' 的內容寫入至 'HW6' 分頁！


In [9]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])

# === 參數：指定要查的星期 ===
# 可填 "一","二","三","四","五","六","日"，或用英文 "Mon" ... "Sun"
TARGET_WEEKDAY = "三"  # 例：查星期三的課

# === 小工具函式 ===
TW_TZ = pytz.timezone(TIMEZONE)

week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
week_map_en = {"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

In [10]:
def normalize_weekday(w):
    if w in week_map_tw: return week_map_tw[w]
    if w in week_map_en: return week_map_en[w]
    raise ValueError("星期請用：一~日 或 Mon~Sun")

In [11]:
def ensure_datetime(s):
    try:
        return pd.to_datetime(s).date()
    except Exception:
        return pd.NaT

In [12]:
def week_monday(any_date):
    # 回到本週一
    return any_date - timedelta(days=any_date.weekday())

In [13]:
def date_range_this_week(today=None):
    # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
    # 這裡的 dt 是指向 datetime.datetime 類別 (來自 from datetime import datetime as dt)
    now = dt.now(TW_TZ).date() if today is None else today

    mon = week_monday(now)
    # 這裡的 timedelta 是指向單獨匯入的 timedelta 類別
    sun = mon + timedelta(days=6)
    return mon, sun

In [14]:
def summarize_courses(day_df):
    if day_df.empty: return "本日無課"
    parts = []
    for _, r in day_df.iterrows():
        parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
    return "；".join(parts)

In [15]:
# === 過濾指定星期的課程（自動只抓「本週」） ===
df["日期"] = df["日期"].apply(ensure_datetime)
df = df.dropna(subset=["日期"])

target_wd = normalize_weekday(TARGET_WEEKDAY)
this_mon, this_sun = date_range_this_week()

mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])

# === 顯示查詢結果（指定星期的所有課） ===
display(Markdown(f"### 本週（{this_mon} ~ {this_sun}）星期{TARGET_WEEKDAY}課程"))
if today_df.empty:
    display(Markdown("> 本日無課"))
else:
    display(today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]])

### 本週（2025-11-10 ~ 2025-11-16）星期三課程

,日期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
105,2025-11-12,3D列印與創新應用,10:20,12:10,誠102,,,


In [16]:
# === 產生一句「行前提醒」（擴充系統智能版） ===

# 方案A：不接模型的簡單規則版（作為 Fallback）
def make_one_line_reminder_simple(rows):
    """簡單規則版本：只列出攜帶品和先讀章節"""
    if rows.empty: return "本日無課，不需準備。"
    items = []
    readings = []
    for _, r in rows.iterrows():
        # 這裡使用 .get() 和 str() 確保安全訪問，即使欄位為空或缺失
        if str(r.get("攜帶品", "")).strip():
            items.append(str(r["攜帶品"]).strip())
        if str(r.get("先讀章節", "")).strip():
            readings.append(str(r["先讀章節"]).strip())

    # 確保唯一性並排序
    items_txt = "；".join(sorted(list(set(items)))) if items else "一般上課用品"
    read_txt  = "；".join(sorted(list(set(readings)))) if readings else "無指定預習"

    return f"請攜帶：{items_txt}；預習：{read_txt}。"

In [17]:
# 方案B：使用 Gemini 模型生成（擴充系統智能）
def make_one_line_reminder(rows):
    """使用 Gemini 模型生成行前提醒"""
    if rows.empty:
        return make_one_line_reminder_simple(rows)

    print("🤖 正在使用 Gemini 2.5 Flash 擴充智能生成行前提醒...")

    # 1. 準備給模型看的課程資訊（將 DataFrame 轉換為文字描述）
    course_list = "本日有以下課程：\n"
    for _, r in rows.iterrows():
        # 格式化日期為 'YYYY/MM/DD 星期X'
        date_str = f"{r['日期'].strftime('%Y/%m/%d')}"

        detail = (
            f"  - 課程: {r['課程名稱']}, 時間: {r['時間(起)']}-{r['時間(迄)']}, 地點: {r['地點']}"
        )
        # 由於原始 PDF 沒資料，這裡假設使用者可能會手動在 Google Sheet 補上
        if r.get("攜帶品", "").strip():
            detail += f", 攜帶品: {r['攜帶品'].strip()}"
        if r.get("先讀章節", "").strip():
            detail += f", 預習: {r['先讀章節'].strip()}"
        if r.get("備註", "").strip():
            detail += f", 備註: {r['備註'].strip()}"

        course_list += detail + "\n"

    # 2. 準備 Prompt
    prompt = f"""
    您是一位貼心的課程行前 AI 助理，專門為使用者準備上課提醒。
    請根據以下課程資訊，用**一句話**在繁體中文中生成一個簡潔、有條理的行前提醒。
    請綜合所有課程的「攜帶品」與「先讀章節」，以流暢的方式呈現。
    如果有多項指定，請用頓號（、）或分號（；）連接。
    如果沒有特別的攜帶品或先讀章節，請以輕鬆的方式（如「別忘了帶筆記本和筆」）作為替代提醒。
    請不要包含任何標頭、開頭語、或結尾標點符號 (如句號)，讓它像一個標題或口號。

    本日課程資訊：
    {course_list}
    """

    # 3. 呼叫 Gemini (使用較快的 Flash 模型)
    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(prompt)

        # 清理並返回模型結果
        # 移除常見的結尾標點符號
        return response.text.strip().replace("。", "").replace("！", "").replace("～", "")
    except Exception as e:
        print(f"⚠️ Gemini 模型生成提醒時發生錯誤，改用簡易規則。錯誤：{e}")
        # Fallback
        return make_one_line_reminder_simple(rows)

In [19]:
# ===================================================
# 👇【優化修改點】：改為呼叫最快的「簡單規則版」
# ===================================================
one_line = make_one_line_reminder_simple(today_df)
# ===================================================

display(Markdown("#### 行前提醒（一句話 - AI 擴充智能生成）"))
display(Markdown(f"> {one_line}"))

# === 回寫「本週重點」分頁 ===
# 若該分頁不存在就自動建立
SHEET_WEEKLY = "本週重點"
try:
    wk = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
except gspread.exceptions.WorksheetNotFound:
    wk = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

# 組成回寫資料：可以一週寫一列
summary_text = summarize_courses(df[mask_week].sort_values(["日期","時間(起)"]))
now_str = dt.now(TW_TZ).strftime("%Y-%m-%d %H:%M")          # ✅ 正確：直接使用 dt.now()
row = {
    "週次起訖": f"{this_mon} ~ {this_sun}",
    "查詢星期": f"星期{TARGET_WEEKDAY}",
    "當日課程列表": summarize_courses(today_df),
    "行前提醒一句話": one_line,
    "更新時間": now_str
}

# 追加寫入（保留歷史）
existing = pd.DataFrame(wk.get_all_records())
outdf = pd.concat([existing, pd.DataFrame([row], columns=row.keys())], ignore_index=True) if not existing.empty else pd.DataFrame([row])

wk.clear()
set_with_dataframe(wk, outdf)

display(Markdown("✅ 已回寫到 **本週重點** 分頁（保留歷史列）。"))

#### 行前提醒（一句話 - AI 擴充智能生成）

> 請攜帶：一般上課用品；預習：無指定預習。

✅ 已回寫到 **本週重點** 分頁（保留歷史列）。

In [20]:
# === 加入 Gradio 編輯面板 ===
def save_edited_reminder(edited_text):
    """讓使用者編輯後回存到 Google Sheet"""
    from datetime import datetime
    import pytz
    tz = pytz.timezone(TIMEZONE)
    now_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M")

    # 重新讀取目前的分頁資料
    try:
        wk_recheck = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
    except gspread.exceptions.WorksheetNotFound:
        wk_recheck = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

    existing = pd.DataFrame(wk_recheck.get_all_records())
    # 追加更新列
    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{TARGET_WEEKDAY}",
        "當日課程列表": summarize_courses(today_df),
        "更新時間": now_str
    }
    outdf = pd.concat([existing, pd.DataFrame([row], columns=row.keys())], ignore_index=True)
    wk_recheck.clear()
    set_with_dataframe(wk_recheck, outdf)
    return f"✅ 已更新至 Google Sheet『本週重點』分頁，目前共 {len(outdf)} 筆。"

In [21]:
# Gradio 介面
with gr.Blocks(title="行前提醒編輯器") as demo:
    gr.Markdown("## ✏️ 行前提醒（可編輯並回存至 Google Sheet）")
    gr.Markdown("這裡顯示的是 **AI 擴充智能** 生成的行前提醒。您可以修改後按下『提交』回存。")
    reminder_box = gr.Textbox(label="行前提醒一句話", value=one_line, lines=3)
    save_btn = gr.Button("💾 提交並回存")
    status = gr.Markdown()

    save_btn.click(save_edited_reminder, inputs=[reminder_box], outputs=[status])

In [22]:
demo.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b2ec83ffac785ae9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
